# Algoritmo Genético

- Como usuário, quero um pacote que eu possa interagir de modo a executar o algoritmo genético de modo inteiro ou passo a passo.

- Como usuário, quero poder fornecer ou não os números aleatórios, de modo a poder comparar as diversas possibilidades de ajuste que o algoritmo permite.

- Como usuário, quero determinar a quantidade de cromossomos no banco de cromossomos principal e a quantidade cromossomos filhos a serem criados, bem como o número de mutantes por filho.



In [ ]:
import numpy as np
import pandas as pd
import random as rdn
import unittest
from IPython.core.debugger import set_trace as db
import collections

In [ ]:
#Permite armazenar os aleatório gerados em um lista ou adicionar valores que serão usados em outras funções
class RandomX:
    def __init__(self):
        self.list = []
        self._index_n = 0 #index dos números lidos através do método next_num
        self._index_r = 0 #index dos números fornecidos pelo método random
    
    def add(self,nums):
        if isinstance(nums, collections.Iterable):
            self.list.extend(nums)
        else:
            self.list.append(nums)
    
    def _generate_random(self,minimo,maximo,count,isint):
        if isint == False:
            l = np.random.uniform(low=minimo, high=maximo,size=count)
        else:
            l = np.random.randint(low=minimo, high=maximo,size=count)
        self.list.extend(l)
        if count == 1:
            return l[0]
        return l
    
    def _get_nums(self,idx,count):
        if idx >= len(self.list):
            return None 
        l = self.list[idx:(idx + count)]
        if count == 1:
            return l[0]
        return l
    
    def random(self,minimo,maximo,count,isint=False):                 
        if len(self.list) >  self._index_r:
            r = self._get_nums(self._index_r,count)
        else:    
            r = self._generate_random(minimo,maximo,count,isint)
        self._index_r += count
        return r
            
    def next_num(self,count=1):
        r = self._get_nums(self._index_n,count)        
        self._index_n += count
        return r    
  
    
rx = RandomX()
l = rx.random(-1,1,19)
print(l)
print(rx.list)
print(rx.next_num())
print(rx.next_num(3))
print(rx.next_num(5))
print(rx.next_num(15))

## Cromossomo

Responsável por armazenar as informações essenciais do  cromossomo e outras necessárias à análise

In [ ]:
class Cromossomo:
    _id = 0
    #Método de classe para reiniciar a contagem dos id's
    @classmethod
    def begin(cls):
        cls._id = 0
        
    def __init__(self,genes, fit, geracao): 
        Cromossomo._id = Cromossomo._id + 1
        self._id = Cromossomo._id        
        self._genes = tuple(genes)
        self._fit = fit
        self._geracao_inicial = geracao
        self._geracao_final = None
        
    @property
    def id(self):
        return self._id
    
    @property
    def genes(self):
        return self._genes   
        
    @property
    def fit(self):
        return self._fit  
    
    @fit.setter
    def fit(self,value):
        self._fit = value
    
    @property
    def geracao_inicial(self):
        return self._geracao_inicial 
    
    @property
    def geracao_final(self):
        return self._geracao_final
    
    #Aceita apenas números inteiros como valor e
    #não permite que valor seja alterado
    @geracao_final.setter
    def geracao_final(self,value):
        if isinstance(value,int) and self._geracao_final is None:
            self._geracao_final = value  
        
    def __hash__(self):
        return hash(self._id)    
    
    def __eq__(self, other):
        return self.genes == other.genes and self.fit == other.fit
    
    """
    Método chamado quando usa a função print()
    Exemplo: 
        genes = np.arange(0.01, 0.10, 0.01) 
        fit = 0.456
        geracao_inicial = 1
        c = Cromossomo(genes, fit , geracao_inicial)
        print(c)
        >>Cromo Id: 1
        >>    Genes: (0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.06999999999999999, 0.08, 0.09) / <class 'tuple'>
        >>    Fit: 0.456
        >>    Geracao Inicial: 1
        >>    Geracao Final: None
    """
    def __str__(self):
        s1 = 'Cromo Id: {}'.format(self.id)
        s2 = '    Genes: {} / {}'.format(self.genes,type(self.genes))
        s3 = '    Fit: {}'.format(self.fit)
        s4 = '    Geracao Inicial: {}'.format( self.geracao_inicial)
        s5 = '    Geracao Final: {}'.format( self.geracao_final)
        return '\n'.join([s1, s2, s3 , s4 , s5])

### Teste de Cromossomos

In [ ]:
# Exemplo de criação de um cromossomo e acesso aos seus métodos
Cromossomo.begin()

def print_cromo(c):
    def _print_cromo(c):
        print('Cromo Id:', c.id)
        print('    Genes: {} / {}'.format(c.genes,type(c.genes)))
        print('    Fit: ', c.fit)
        print('    Geracao Inicial: ', c.geracao_inicial)
        print('    Geracao Final: ', c.geracao_final)
        
    if isinstance(c, collections.Iterable):
        for i in c:
            _print_cromo(i)
    else:
        _print_cromo(c)

genes = np.arange(0.01, 0.10, 0.01) 
fit = 0.456
geracao_inicial = 1
c = Cromossomo(genes, fit , geracao_inicial)
print(c)

In [ ]:
c.fit = 5

In [ ]:
genes = np.arange(0.01, 0.10, 0.01) 
fit = 0.456
geracao_inicial = 1
c1 = Cromossomo(genes, fit , geracao_inicial)

geracao_inicial = 1
c2 = Cromossomo(genes, fit , geracao_inicial)

print(c1)
print(c1)

print('\n\n c1 é igual a c2? ',c1 == c2)

genes = np.arange(0.2, 0.3, 0.01) 
fit = 0.456
geracao_inicial = 1
c3 = Cromossomo(genes, fit , geracao_inicial)

print(c3)
print('\n\n c1 é igual a c3? ',c1 == c3)

In [ ]:
class TestCromo(unittest.TestCase):
    def test_ctor(self):
        Cromossomo.begin()
        g = (np.arange(0.01, 0.11, 0.01))
        
        c = Cromossomo(g,0.5,2)
        self.assertTupleEqual(tuple(g),c.genes)
        self.assertEqual(1,c.id)
        self.assertEqual(2,c.geracao_inicial)
        
        c.geracao_final=15
        self.assertEqual(15,c.geracao_final)
    
    def test_id_sequence(self):
        Cromossomo.begin()
        g = (np.arange(0.01, 0.11, 0.01))
        
        c1 = Cromossomo(g,0.5,2)
        self.assertTupleEqual(tuple(g),c1.genes)
        self.assertEqual(1,c1.id) 
        
        c2 = Cromossomo(g,0.5,1)
        self.assertEqual(2,c2.id)
        
        c3 = Cromossomo(g,0.5,1)
        self.assertEqual(3,c3.id)
        
    def test_id_sequence_multi(self):
        Cromossomo.begin()
        g = np.arange(0.01, 0.11, 0.01)
        
        for i in range(1,1000):
            c = Cromossomo(g,0.5,1)
            self.assertTupleEqual(tuple(g),c.genes)
            self.assertEqual(i,c.id)    
     
    def test_equal(self):
        
        g = np.arange(0.01, 0.11, 0.01)
        c = Cromossomo(g,0.5,1)
       #Testa igualdade positiva para genes e fitness iguais 
        c2 = Cromossomo(g,0.5,1)        
        self.assertTrue(c2 == c)
        
       #Testa igualdade negativa para genes diferentes e fitness iguais (impossível na prática!!!)
        g = np.arange(0.02, 0.21, 0.01)
        c2 = Cromossomo(g,0.5,1)
        self.assertFalse(c2 == c)
        
       #Testa igualdade negativa para genes iguais e fitness diferentes (impossível na prática!!!)
        c2 = Cromossomo(g,0.3,2)        
        self.assertFalse(c2 == c)         
       
            
if __name__ == '__main__':
    unittest.main(argv=['TestCromo'], exit=False)        

## Banco de Cromossomos

Responsabilidades da classe BancoCromos:
- Armazenar uma lista de cromossomos
- Permite adicionar um ou mais cromossomos
- Permite remover um ou mais dos piores cromossomos
- Permite remover um ou mais dos melhores cromossomos
- Permite obter por sorteio um ou mais cromossomos, tendo o fitness como peso
- Permite obter por sorteio ou não um ou mais genes do banco

In [ ]:
class BancoCromos:
    def __init__(self, random_callback):
        self._random_callback = random_callback        
        self._cromos =[]
        
    def add(self,c):
        if isinstance(c, collections.Iterable):
            self._cromos.extend(c)
        else:
            self._cromos.append(c)
    
    def _remove(self,cromo_list,count):
         #cria um set dos "count" cromos piores
        cromo_list = set(cromo_list[:count])   
        #remove os "cromos piores" do banco
        self._cromos = list(set(self._cromos) - cromo_list)
        if len(cromo_list) == 1:
            return cromo_list.pop()
        return cromo_list
    
    def remove_worst(self,count=1):
        #cria uma nova lista com os cromos ordenados pelo fit em ordem decrescente
        l = sorted(self._cromos, key=lambda x: x.fit)
        return self._remove(l,count)
    
    def remove_best(self,count=1):
        l = sorted(self._cromos, key=lambda x: x.fit, reverse=True)
        return self._remove(l,count)
   
    @property
    def best_fit(self):
        l = sorted(self._cromos, key=lambda x: x.fit, reverse=True)
        return l[0].fit
    
    @property
    def best_cromos(self,count=1):
        l = sorted(self._cromos, key=lambda x: x.fit, reverse=True)
        return l[:1]
    
    def random_by_fit(self,count=1):
        r = [] 
        cs = self._cromos.copy()
        
        total_fits = 0
        for c in self._cromos:
            total_fits += c.fit
                
        while len(r)<count:                
            fit_rdn = self._random_callback(0,total_fits,1)
            fit_acumulado = 0                   

            for c in cs:
                fit_acumulado += c.fit
                if fit_acumulado >= fit_rdn:
                    r.append(c)
                    cs.remove(c)
                    total_fits -= c.fit
                    break
     
        if count == 1:
            return r[0]
        return r                   
                
    def cromo(self,idx=None):
        if idx is None:
            return self._cromos
        return self._cromos[idx]
    
    def remover_todos(self):
        cromos = self._cromos
        self._cromos = []
        return cromos
        
    @property
    def cromo_count(self):
        return len(self._cromos) 
    
    @property
    def _cromo_gene_count(self):
        return len(self._cromos[0].genes) 
    
    # retorna 1 gene do banco
    def _gene(self,idx):
        gene_idx = idx % self._cromo_gene_count
        cromo_idx = idx // self._cromo_gene_count
        return self._cromos[cromo_idx].genes[gene_idx]
    
    # Retorna 1 ou mais genes do banco
    # Se o parâmetro "genes" é um inteiro, é porque foi passada a quantidade de genes a serem retornados, 
    #escolhidos aleatoriamente
    # Se o parâmetro "genes" não é um inteiro (é uma lista), retorna uma lista com os genes solicitados, 
    #considerando cada item da lista como índice de um gene
    def genes(self,genes=1):
        
        if isinstance(genes, int):
            total_genes = len(self._cromos) * self._cromo_gene_count
            genes_index = self._random_callback(0,total_genes-1,genes,True)
        else:        
            genes_index = genes
            
        if genes == 1:
            return self._gene(genes_index)
        
        genes_list = []
        for i in genes_index:
            g = self._gene(i)
            genes_list.append(g)
        return genes_list
    
    #Calcula o desvio padrão dos fitness dos vários cromossomos 
    def calc_fit_dp(self):
        l = []
        for c in self._cromos:
            l.append(c.fit)
            
        l = np.array(l)
        return np.std(l)

In [ ]:
class TestBancoCromos(unittest.TestCase):     
    
    def test_add_and_getcromo(self):
        rx = RandomX()
        bc = BancoCromos(rx.random)
        
        c_count = 19
        rnd_genes = lambda: rx.random(-1,1,c_count)
        
        
        c1 = Cromossomo(rnd_genes(),1,1)
        c2 = Cromossomo(rnd_genes(),2,1)
        c3 = Cromossomo(rnd_genes(),3,1)
        c4 = Cromossomo(rnd_genes(),4,1)
        c5 = Cromossomo(rnd_genes(),5,1)
        c6 = Cromossomo(rnd_genes(),6,1)
        
        bc.add(c1)
        self.assertEqual(1,bc.cromo_count)
        self.assertEqual(c1,bc.cromo(0))
        
        bc.add(c2)
        self.assertEqual(2,bc.cromo_count)
        self.assertEqual(c2,bc.cromo(1))
        
        bc.add(c3)
        self.assertEqual(3,bc.cromo_count)
        self.assertEqual(c3,bc.cromo(2))
        
        bc.add(c4)
        self.assertEqual(4,bc.cromo_count)
        self.assertEqual(c4,bc.cromo(3))
        
        bc.add(c5)
        self.assertEqual(5,bc.cromo_count)
        self.assertEqual(c5,bc.cromo(4))
        
        bc.add(c6)
        self.assertEqual(6,bc.cromo_count)
        self.assertEqual(c6,bc.cromo(5))
        
    def test_random_by_fit_single(self):
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)       
        # Cromo 1: -  0 à 1
        # Cromo 2: -  2 à 3
        # Cromo 3: -  4 à 6
        # Cromo 4: -  7 à 10
        # Cromo 5: - 11 à 15
        # Cromo 6: - 16 à 21
        
        rx.add(0) #Induzo o método random_by_fit a escolher o cromossomo 1
        cx = bc.random_by_fit()
        self.assertTrue (isinstance(cx,Cromossomo),'Não retornou um cromossomo ao pedir apenas 1 cromossomo')         
        self.assertEqual(cx,bc.cromo(0),"Cromo 1 - Erro limite inferior")
        
        rx.add(1) #Induzo o método random_by_fit a escolher o cromossomo 1
        cx = bc.random_by_fit()
        self.assertTrue (isinstance(cx,Cromossomo),'Não retornou um cromossomo ao pedir apenas 1 cromossomo')        
        self.assertEqual(cx,bc.cromo(0),"Cromo 1 - Erro limite superior")
        
        rx.add(2) #Induzo o método random_by_fit a escolher o cromossomo 2 pelo limite inferior
        cx = bc.random_by_fit()
        self.assertTrue (isinstance(cx,Cromossomo),'Não retornou um cromossomo ao pedir apenas 1 cromossomo')        
        self.assertEqual(cx,bc.cromo(1),"Cromo 2 - Erro limite inferior")
        
        rx.add(3) #Induzo o método random_by_fit a escolher o cromossomo 2 pelo limite superior
        cx = bc.random_by_fit()
        self.assertTrue (isinstance(cx,Cromossomo),'Não retornou um cromossomo ao pedir apenas 1 cromossomo')        
        self.assertEqual(cx,bc.cromo(1),"Cromo 2 - Erro limite superior")
        
        rx.add(21) #Induzo o método random_by_fit a escolher o cromossomo 6 pelo limite superior
        cx = bc.random_by_fit()
        self.assertTrue (isinstance(cx,Cromossomo),'Não retornou um cromossomo ao pedir apenas 1 cromossomo')        
        self.assertEqual(cx,bc.cromo(5),"Cromo 6 - Erro limite superior")
        
    def test_random_by_fit_multi(self):
        rx = RandomX()
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        bc = BancoCromos(rx.random) 
        
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)       
        # Cromo 1: -  0 à 1
        # Cromo 2: -  2 à 3
        # Cromo 3: -  4 à 6
        # Cromo 4: -  7 à 10
        # Cromo 5: - 11 à 15
        # Cromo 6: - 16 à 21
        
        rx.add(0)
        rx.add(1)
        cl = bc.random_by_fit(2)
        self.assertTrue (isinstance(cl,list),'Não retornou uma lista ao pedir apenas 2 cromossomos') 
        self.assertEqual (2,len(cl),'Não retornou quantidade certa de cromossomos') 
        self.assertEqual(bc.cromo(0),cl[0],"Cromo 1 - não encontrado (T1)")
        self.assertEqual(bc.cromo(1),cl[1],"Cromo 2 - não encontrado (T2)")  
        
        rx.add(3)
        rx.add(0)
        #db()
        cl = bc.random_by_fit(2)
        self.assertTrue (isinstance(cl,list),'Não retornou uma lista ao pedir apenas 2 cromossomos') 
        self.assertEqual (2,len(cl),'Não retornou quantidade certa de cromossomos')         
        self.assertEqual(bc.cromo(1),cl[0],"Cromo 2 - não encontrado (T3)")
        self.assertEqual(bc.cromo(0),cl[1],"Cromo 1 - não encontrado(T4)")   
        
        rx.add(8)
        rx.add(13)
        cl = bc.random_by_fit(2)
        self.assertTrue (isinstance(cl,list),'Não retornou uma lista ao pedir apenas 2 cromossomos') 
        self.assertEqual (2,len(cl),'Não retornou quantidade certa de cromossomos') 
        self.assertEqual(bc.cromo(3),cl[0],"Cromo 4 - não encontrado (T5)")
        self.assertEqual(bc.cromo(5),cl[1],"Cromo 6 - não encontrado (T6)")
        
        
        rx.add(8)
        rx.add(13)
        rx.add(5)
        cl = bc.random_by_fit(3)
        self.assertTrue (isinstance(cl,list),'Não retornou uma lista ao pedir apenas 3 cromossomos') 
        self.assertEqual (3,len(cl),'Não retornou quantidade certa de cromossomos') 
        self.assertEqual(bc.cromo(3),cl[0],"Cromo 4 - não encontrado (T7)")
        self.assertEqual(bc.cromo(5),cl[1],"Cromo 6 - não encontrado (T8)")
        self.assertEqual(bc.cromo(2),cl[2],"Cromo 6 - não encontrado (T9)") 
        
    def test_remove_worst(self):   
        rx = RandomX()
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        bc = BancoCromos(rx.random)         
        g_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,g_count)
            c = Cromossomo(genes,7-i,1)
            bc.add(c)  
        # CROMO    -  FIT
        # Cromo 1: -  6
        # Cromo 2: -  5
        # Cromo 3: -  4
        # Cromo 4: -  3
        # Cromo 5: -  2
        # Cromo 6: -  1
        
        self.assertEqual(6,bc.cromo_count,'Quantidade de cromossomos incorreta após a criação do banco') 
        
        #Salva lista de cromossomos para comparar depois
        cexp_lst = bc._cromos    
        
        # Teste removendo o pior cromossomos 
        c = bc.remove_worst()
        self.assertTrue (isinstance(c,Cromossomo),'Não retornou um obj Cromossomo ao pedir apenas 1 cromossomo') 
        self.assertEqual(cexp_lst[5],c,"Cromo 5 não encontrado") 
        self.assertEqual(5,bc.cromo_count,'Quantidade de cromossomos incorreta após remover 1') 
        
       # Teste removendo os 2 piores cromossomos
        #db()
        c = bc.remove_worst(2) 
        self.assertEqual(3,bc.cromo_count,'Quantidade de cromossomos incorreta após remover mais 2 cromos')
        self.assertTrue (isinstance(c,set),'Não retornou uma lista ao pedir apenas 2 cromossomos') 
        self.assertEqual (2,len(c),'Não retornou quantidade certa de cromossomos') 
        c = sorted(list(c), key=lambda x: x.fit)
        self.assertEqual(cexp_lst[4],c[0],"Cromo 4 não encontrado")     
        self.assertEqual(cexp_lst[3],c[1],"Cromo 3 não encontrado") 
        
    def test_remove_best(self):   
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)  
        
        # CROMO    -  FIT
        # Cromo 1: -  1
        # Cromo 2: -  2
        # Cromo 3: -  3
        # Cromo 4: -  4
        # Cromo 5: -  5
        # Cromo 6: -  6
        
        self.assertEqual(6,bc.cromo_count,'Quantidade de cromossomos incorreta após a criação do banco') 
        
        #Salva lista de cromossomos para comparar depois
        cexp_lst = bc._cromos    
        
        # Teste removendo o melhor cromossomos 
        c = bc.remove_best()
        self.assertTrue (isinstance(c,Cromossomo),'Não retornou um obj Cromossomo ao pedir apenas 1 cromossomo') 
        self.assertEqual(cexp_lst[5],c,"Cromo 5 não encontrado (T1)") 
        self.assertEqual(5,bc.cromo_count,'Quantidade de cromossomos incorreta após remover 1') 
        
       # Teste removendo os 2 melhores cromossomos
        #db()
        c = bc.remove_best(2) 
        self.assertEqual(3,bc.cromo_count,'Quantidade de cromossomos incorreta após remover mais 2 cromos')
        self.assertTrue (isinstance(c,set),'Não retornou uma lista ao pedir apenas 2 cromossomos') 
        self.assertEqual (2,len(c),'Não retornou quantidade certa de cromossomos') 
        c = sorted(list(c), key=lambda x: x.fit,reverse=True)
        self.assertEqual(cexp_lst[4],c[0],"Cromo 4 não encontrado")     
        self.assertEqual(cexp_lst[3],c[1],"Cromo 3 não encontrado") 
    
    def test_best_fit(self):
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)  
            
        self.assertEqual(6,bc.best_fit,"Melhor fit não encontrado")         
    
    def test_genes(self):  
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        c_count = 20       
        for i in range(1,6):
            genes = np.arange(i*100+1,i*100+c_count+1)
            c = Cromossomo(genes,i,1)
            bc.add(c)        
        
        # CROMO    -  FIT
        # Cromo 1: -  geness 101 a 120
        # Cromo 2: -  geness 201 a 220
        # Cromo 3: -  geness 301 a 320
        # Cromo 4: -  geness 401 a 420
        # Cromo 5: -  geness 501 a 520        
        
        rx.add(0)
        g = bc.genes()    
        self.assertEqual(101,g)
        
        rx.add(19)
        g = bc.genes()    
        self.assertEqual(120,g)        
        
        rx.add(49)
        g = bc.genes()    
        self.assertEqual(310,g)  
        
        rx.add(99)
        g = bc.genes()    
        self.assertEqual(520,g)
        
        rx.add(49)
        rx.add(99)
        #db()
        g = bc.genes(2)   
        self.assertEqual(310,g[0])         
        self.assertEqual(520,g[1])
        
    def test_calc_fit_dp(self):        
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)  
            
        self.assertEqual(1.707825127659933,bc.calc_fit_dp())
    
    def test_remover_todos(self):
        rx = RandomX()
        bc = BancoCromos(rx.random) 
        # Cria um banco de cromossomos com 6 cromossomos com 19 genes
        c_count = 19 
        for i in range(1,7):
            genes = rx.random(-1,1,c_count)
            c = Cromossomo(genes,i,1)
            bc.add(c)
    
        cromos_exp = bc.cromo()        
        cromos = bc.remover_todos() 
        # Asserta que retornou todos cromossomos que estavam dentro do banco
        self.assertEqual(cromos_exp,cromos,"Coleção de cromossomos removidos não é igual à coleção pré-existente")
    
        cromos_exp = bc.cromo() 
        # Asserta que retornou todos cromossomos que estavam dentro do banco
        self.assertEqual(cromos_exp,[],"O banco de cromossomos não ficou vazio")
    
if __name__ == '__main__':
    unittest.main(argv=['TestBancoCromos'], exit=False)          

## Fábrica de cromossomos

In [ ]:
class CromoFactory:
    def __init__(self, genes_count, calcfit_callback, random_callback):
        self._genes_count = genes_count
        self._calcfit_callback = calcfit_callback
        self._random_callback = random_callback        
        self._geracao = 0
       
    def next_geracao(self):
        self._geracao += 1
    
    def _criar_cromo(self,genes):
        f = self._calcfit_callback(genes)
        return Cromossomo(genes,f,self._geracao)
    
    def _sortear_genes(self):
        return self._random_callback(-1,1,self._genes_count)
        
    def criar_cromo_random(self,count=1):
        if count == 1:
            gs = self._sortear_genes()
            return self._criar_cromo(gs)
        lc = []
        for i in range(count):
            gs = self._sortear_genes()
            c = self._criar_cromo(gs)
            lc.append(c)
        return lc 
        
    def criar_cromo_user(self,genes):      
        return self._criar_cromo(genes)
    
       
    def criar_cromo_filho(self, cromo_pai:Cromossomo, cromo_mae:Cromossomo, cross_point=None):        
        if cross_point is None:
            cross_point = self._random_callback(1,self._genes_count,1,True)           
                  
        gs = []
        gs.extend(cromo_pai.genes[:cross_point])
        gs.extend(cromo_mae.genes[cross_point:])       
        return self._criar_cromo(gs)
        
    def criar_cromo_mutante(self,cromo:Cromossomo,genes):
        if not isinstance(genes,collections.Iterable):
            genes = [genes]
        gs = list(cromo.genes)
        gpos = self._random_callback(0,self._genes_count-1,len(genes),True)
        if not isinstance(gpos,collections.Iterable):
            gpos = [gpos]
            
        j = 0
        for i in gpos:            
            gs[i] = genes[j]
            j += 1
   
        return self._criar_cromo(gs)

### Testes da Fábrica de Cromossomos

In [ ]:
class TestCromoFactory(unittest.TestCase):     
    
    def test_rdn(self):
        rx = RandomX()       
        calc_fit = lambda c: 2
        g_count = 19
        cf = CromoFactory(g_count,calc_fit,rx.random)        
        c_act = cf.criar_cromo_random()
        g_exp = tuple(rx.next_num(g_count))
       
        self.assertEqual(g_exp,c_act.genes)
        self.assertEqual(2,c_act.fit)      
        
    def test_filho_com_cross_ext(self):
        rx = RandomX()   
        calc_fit = lambda c: 2
        g_count = 19
        cf = CromoFactory(g_count,calc_fit,rx.random)
        
        c_pai = cf.criar_cromo_random()
        c_mae = cf.criar_cromo_random()
        # Cria filho fornecendo pai, mãe e ponto de corte
        cross=8
        c_filho = cf.criar_cromo_filho(cromo_pai=c_pai, cromo_mae=c_mae, cross_point=cross)
        self.assertEqual(c_pai.genes[:cross],c_filho.genes[:cross])
        self.assertEqual(c_mae.genes[cross:],c_filho.genes[cross:])     
        
    def test_filho_com_cross_rdn(self):
        rx = RandomX()   
        calc_fit = lambda c: 2
        g_count = 19
        cf = CromoFactory(g_count,calc_fit,rx.random)   
        
        c_pai = cf.criar_cromo_random()
        c_mae = cf.criar_cromo_random()
        
        # Cria filho fornecendo pai, mãe e ponto de corte random
        cross=4
        rx.add(cross)
        c_filho = cf.criar_cromo_filho(cromo_pai=c_pai, cromo_mae=c_mae)
        self.assertEqual(c_pai.genes[:cross],c_filho.genes[:cross])
        self.assertEqual(c_mae.genes[cross:],c_filho.genes[cross:])    
        
    def test_cromo_mut(self):
        rx = RandomX()   
        calc_fit = lambda c: 2
        g_count = 19
        cf = CromoFactory(g_count,calc_fit,rx.random)   
        
        c = cf.criar_cromo_random()
      
        #Testa com gene de mutação na posição 0
        gl = .001 
        gpos = 0
        rx.add(gpos)
        #db()
        cx = cf.criar_cromo_mutante(c,gl)
    
       #Testa com gene de mutação na posição 0
        gl = [.001] 
        gpos = 0
        rx.add(gpos)
        cx = cf.criar_cromo_mutante(c,gl)
        
        self.assertEqual(gl[0],cx.genes[gpos])
        self.assertEqual(c.genes[gpos+1:],cx.genes[gpos+1:])    
                
       #Testa com gene de mutação na posição 7  
        gl = [.007] 
        gpos = 7
        rx.add(gpos)
        cx = cf.criar_cromo_mutante(c,gl)
        
        self.assertEqual(gl[0],cx.genes[gpos])
        self.assertEqual(c.genes[:gpos],cx.genes[:gpos])    
        self.assertEqual(c.genes[gpos+1:],cx.genes[gpos+1:])    
        
       #Testa com gene de mutação na posição 19  
        gl = [.009] 
        gpos = g_count -1
        rx.add(gpos)
        cx = cf.criar_cromo_mutante(c,gl)
        
        self.assertEqual(gl[0],cx.genes[gpos])
        self.assertEqual(c.genes[:gpos],cx.genes[:gpos])              
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) 

  ## Algoritmo Genético

In [ ]:
class Gapy:
    # Todo: Criar uma classe configuradora, usando o padrão Builder, do Gapy por meio de Interface fluente em Python
    # https://www.youtube.com/watch?v=NRoMlyBKMeE
    def _preparar_rodada(self):
        self._mb = BancoCromos(self._rdn_cb) #banco principal com os melhores cromossomos
        self._rb = BancoCromos(self._rdn_cb) #banco de reprodução de cromossomos
        self._bb = BancoCromos(self._rdn_cb) #banco de com os melhores cromossomos por rodada
    
    def __init__(self, calcfit_callback, random_callback,genes_count,soncromos_count = 3, mutcromos_count = 1, taxa_subst = 2):
        Cromossomo.begin()
        self._rdn_cb = random_callback       
        self._melhores_por_rodada = []
        self._preparar_rodada()
        self._factory = CromoFactory(genes_count, calcfit_callback, random_callback)   
        self._soncromos_count = soncromos_count
        self._mutcromos_count = mutcromos_count #total de mutantes para cada filho
        self._cromos_count = soncromos_count + soncromos_count * mutcromos_count    
        self._taxa_subst = taxa_subst        
            
            
            
    def _reproduzir(self):
        c_pais = self._mb.random_by_fit(2)
        c = self._factory.criar_cromo_filho(c_pais[0],c_pais[1])
        self._rb.add(c)
        for i in range(self._mutcromos_count):            
            #Obtém uma lista de genes para mutação
            genes = self._mb.genes(self._mutcromos_count)
            #cria o mutante
            cx = self._factory.criar_cromo_mutante(c,genes)
            #armazena o mutante
            self._rb.add(cx)    
     
    def _criar_geracao(self):
        self._factory.next_geracao()
        #cria os cromos random para completar a quantidade necessária determinada
        i = self._cromos_count - self._mb.cromo_count 
        if i > 0:
            c = self._factory.criar_cromo_random(i)
            self._mb.add(c)
        
        self._reproduzir()
        self._mb.remove_worst(self._taxa_subst)
        cl = self._rb.remove_best(self._taxa_subst)
        self._mb.add(cl)
        self._rb.remover_todos()
        return self._mb.calc_fit_dp()
        
    
    def melhores_cromos(self):
        return self._mb.cromo()    
    
    def executar_por_geracao(self,count = 1):      
        for i in range(count):
            dp = self._criar_geracao()
            #print(dp)
        return dp
   
    def iniciar_nova_rodada(self, usar_melhores = False):
        if self._mb.cromo_count > 0:
            c = self._mb.remove_best(1)
            self._bb.add(c)
            self._mb.remover_todos()
        
        if usar_melhores == True:
            c = self._bb.cromos()
            self._mb.add(c)
        
        #self._preparar_rodada()
    
    def executar_por_rodada(self,count = 1, desvio_padrao=0.01, usar_melhores = False):
        for i in range(count):
            self.iniciar_nova_rodada(usar_melhores)
            while True:
                dp = self._criar_geracao()
                if dp <= desvio_padrao:                
                    break
        
   

## Teste Simples do Algoritmo Genético

In [ ]:
#Função simples de cálculo do fitness
licao = np.array([1,1,1,1,1,0,0,0,0,0])
def calc_fit_simples(t):   
    t = np.array(t)
    r = (t * licao)
    r = np.sum(r)
    return r

In [ ]:
#teste da função de cálculo de fitness
t = np.ones(len(licao))
calc_fit_simples(t)

### Teste de mesa de uma geração

Para testar a classe Gapy

In [ ]:

total_genes = len(licao)
rx = RandomX()
bc1 = BancoCromos(rx.random)
bc2 = BancoCromos(rx.random)
Cromossomo.begin()
factory = CromoFactory(len(licao), calc_fit_simples, rx.random)   

fit_acumulado = 0
for i in range(1, 7):
   #Cria genes decimais: 1º dígito = Id Cromosssomo / 2ª dígito = incremental
    primeiro_gene = i*10    
    ultimo_gene = total_genes + primeiro_gene   
    genes = np.arange( primeiro_gene, ultimo_gene , 1, )
    genes = genes / 100 #tranforma genes em decimais
   #Cria cromossomo com os genes 
    rx.add(genes)
    c = factory.criar_cromo_random()
    bc1.add(c)    
    print("Cromo ",i)
    print("   Genes:",c.genes) 
    print("   Fit:",c.fit)
    fit_acumulado += c.fit
    print("   Fit Acc:",fit_acumulado) 
    
#Filho 1 e Mutante 1
rx.add([11,0.5])
cl = bc1.random_by_fit(2)
print("Pai 1")
print_cromo(cl[0])

print("Mãe 1")
print_cromo(cl[1])

rx.add(5) #Cross no gene 5
cf = factory.criar_cromo_filho(cl[0],cl[1])
print("Filho 1")
print_cromo(cf)

rx.add(30-1)
g = bc1.genes(1) #pega gene 30 e coloca na posição 4
rx.add(5)
cx = factory.criar_cromo_mutante(cf,g)
print("Mutante 1")
print_cromo(cx)

bc2.add([cf,cx])


#Filho 2 e Mutante 2
rx.add([1.5,7-1.5])#seleciona cromos 5 e 2
cl = bc1.random_by_fit(2)
print("Pai 2")
print_cromo(cl[0])

print("Mãe 2")
print_cromo(cl[1])

rx.add(3) #Cross no gene 2
cf = factory.criar_cromo_filho(cl[0],cl[1])
print("Filho 2")
print_cromo(cf)

rx.add(20-1)#
g = bc1.genes(1)
rx.add(2)
cx = factory.criar_cromo_mutante(cf,g)
print("Mutante 2")
print_cromo(cx)

bc2.add([cf,cx])


#Filho 3 e Mutante 3
rx.add([3,5.4-3.3])#seleciona cromos 4 e 3
cl = bc1.random_by_fit(2)
print("Pai 3")
print_cromo(cl[0])

print("Mãe 3")
print_cromo(cl[1])

rx.add(7) #Cross no gene 7
cf = factory.criar_cromo_filho(cl[0],cl[1])
print("Filho 3")
print_cromo(cf)

rx.add(40-1)#
g = bc1.genes(1)
rx.add(7)
cx = factory.criar_cromo_mutante(cf,g)
print("Mutante 3")
print_cromo(cx)

bc2.add([cf,cx])



#Remove piores e substitui por melhores
print("----------------")
print("Remove Piores:")
print("----------------")
cl = list(bc1.remove_worst(2))
print_cromo(cl[0])
print_cromo(cl[1])

print("----------------")
print("Adiciona Melhores:")
print("----------------")

cl = list(bc2.remove_best(2))
print_cromo(cl[0])
print_cromo(cl[1])

bc1.add(cl)
#Imprime lista de todos números usados
print(rx.list)

|Cromo ID | Gene Inicial | Gene Final | Fitness|
|---------|--------------|------------|--------|
|1 |.10|.19|0.6|
|2 |.20|.29|1.1|
|3 |.30|.39|0.6|
|4 |.40|.49|0.6|
|5 |.50|.59|0.6|
|6 |.60|.69|0.6|


In [ ]:
class TestGapySimples(unittest.TestCase):
    def test_geracao_inicial(self):
        rx = RandomX()
        #seta genes dos cromossomos primários
        # c1 ......  .10 à .19
        # c2 ......  .20 à .29
        # c3 ......  .30 à .39
        # c4 ......  .40 à .49
        # c5 ......  .50 à .59
        # c6 ......  .60 à .69        
        
        #Adiciona os mesmos números usados no teste de mesa, para forçar o mesmo resultado
        rx.add([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 11, 0.5, 5, 29, 5, 1.5, 5.5, 3, 19, 2, 3, 2.1000000000000005, 7, 39, 7])  
        
        ga = Gapy(calc_fit_simples,rx.random,len(licao))
        dp = ga.executar_por_geracao()
        for c in ga.melhores_cromos():
            print_cromo(c)
            
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)             

In [ ]:
rx = RandomX()
rx.add([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 11, 0.5, 5, 29, 5, 1.5, 5.5, 3, 19, 2, 3, 2.1000000000000005, 7, 39, 7])  
ga = Gapy(calc_fit_simples,rx.random,len(licao))
dp = ga.executar_por_geracao(1)
print(dp)
for c in ga.melhores_cromos():
    print_cromo(c)

In [ ]:
dp = ga.executar_por_geracao(10)
for c in ga.melhores_cromos():
    print_cromo(c)

## Creditus

### Criação do banco de amostras

In [ ]:
import pandas as pd
    
class Creditus(Gapy):
    def __init__(self, random_callback, 
                    amostra_qtde = 100,
                    cromofilho_qtde = 3, 
                    cromomut_qtde = 1, 
                    taxa_subst = 2):        
       
        #Lê dataset
        self.licoes = pd.read_csv('Creditus-Dados.csv', sep=';',index_col=0)
        self.atributos_qtde = self.licoes.shape[1] - 1 
        
        #Organiza dataset por resultados
        self.licoes = self.licoes.sort_values(by=['Resultado'])
                 
        Gapy.__init__(self, self._calcular_fit, 
                      random_callback,
                      self.atributos_qtde,
                      cromofilho_qtde,                      
                      cromomut_qtde, 
                      taxa_subst)
                 
        self.criar_banco_de_amostras(amostra_qtde)
                 
    def criar_banco_de_amostras(self, qtde):
        #Obtém uma Série com a quantidade de cada tipo de resultado
        resultados = self.licoes.groupby('Resultado').size()

        #Cria uma lista com os índices da lições que irão compor o bando de amostra
        amostra_list_idx = []
        mininimo = 0
        maximo = 0       
        qtde_por_resultado = int(qtde / resultados.shape[0])
        for i in resultados:
            maximo += i 
            amostra_list_idx.extend(np.random.randint(low=mininimo, high=maximo,size=qtde_por_resultado))
            mininimo += i

        #Usa os índices criados para gerar o banco de amostra
        self.amostras = self.licoes.iloc[amostra_list_idx]
        #df.insert(idx, col_name, value) inserir coluna de ajuste (b0)
        #amostra_result = amostra_lics.groupby('Resultado').size()
        #amostra_result
        return self.amostras
    
    def _calcular_fit(self, genes):
        genes = np.array(genes)
        nac = 0
        nic = 0
        ta = 0
        ti = 0
        #db()  
        for i in range(self.amostras.shape[0]):         
            licao = self.amostras.iloc[i]
            lic_attr = licao[:self.atributos_qtde] 
            resultado = licao[-1]
            somaprod =(lic_attr* genes).values.sum()
            if somaprod > 0 and resultado == 'A':
                nac += 1
            elif somaprod < 0 and resultado == 'I':
                nic += 1
            if resultado == 'A':
                ta += 1
            else:
                ti += 1
        fit = (nac*nic)/(ta*ti)
        #fit = (nac+nic)/(ta+ti)
        return fit
    
    def calcular_acerto(self, genes = None):
        
        if genes is None:
            c = self.melhores_cromos()
            genes = c[0].genes
                 
        genes = np.array(genes)
        nac = 0
        nic = 0
        ta = 0
        ti = 0
        #db()  
        for i in range(self.licoes.shape[0]):         
            licao = self.licoes.iloc[i]
            lic_attr = licao[:self.atributos_qtde] 
            
            resultado = licao[-1]
            somaprod =(lic_attr * genes).values.sum()
            if somaprod > 0 and resultado == 'A':
                nac += 1
            elif somaprod < 0 and resultado == 'I':
                nic += 1
            if resultado == 'A':
                ta += 1
            else:
                ti += 1       
        acerto = (nac+nic)/(ta+ti) * 100
        return str(round(acerto,2)) + "%"
    

In [ ]:
rx = RandomX()
cred = Creditus(rx.random,200)
cred.executar_por_rodada(1,0.00001)
cred.calcular_acerto()

In [ ]:
for i in range(6):
    cred.executar_por_rodada(1)
    cred.calcular_acerto()
cred.executar_por_rodada(1,True)    
cred.calcular_acerto()    

In [ ]:
rx = RandomX()
cred20 = Creditus(rx.random,100,20,2,10)
cred20.executar_por_rodada(1)
cred20.calcular_acerto()

In [ ]:
for i in range(1000):
    dp = cred.executar_por_geracao(1)
    c = cred.melhores_cromos()  
    print('Id:', c[0].id, " / Fit: ", c[0].fit, " / Dp: ", dp) 
cred.calcular_acerto()    

In [ ]:
for i in range(1000):
    dp = cred20.executar_por_geracao(1)
    c = cred20.melhores_cromos()  
    print('Id:', c[0].id, " / Fit: ", c[0].fit, " / Dp: ", dp) 
cred20.calcular_acerto()    


In [ ]:
print_cromo(cred20.melhores_cromos())

In [ ]:
#def dbf():
    #db()
rx = RandomX()
credex = Creditus(rx.random)
cred.amostras
#dbf() 

In [ ]:
rx = RandomX()
cred40 = Creditus(rx.random,100,40,1,20)
cred40.executar_por_rodada(1)
cred40.calcular_acerto()

In [ ]:
for i in range(100):
    dp = cred40.executar_por_geracao(1)
    c = cred40.melhores_cromos()  
    print('Id:', c[0].id, " / Fit: ", c[0].fit, " / Dp: ", dp) 
cred40.calcular_acerto() 

### Creditus: função de cálculo do fitness

In [ ]:
#Lê dataset
licoes = pd.read_csv('Creditus-Dados.csv', sep=';',index_col=0)
atributos_qtde = licoes.shape[1] - 1 

#Organiza dataset por resultados
licoes = licoes.sort_values(by=['Resultado'])
        
#Obtém uma Série com a quantidade de cada tipo de resultado
resultados = licoes.groupby('Resultado').size()

#Cria uma lista com os índices da lições que irão compor o bando de amostra
amostra_list_idx = []
mininimo = 0
maximo = 0
qtde = 100
qtde_por_resultado = int(qtde / resultados.shape[0])
for i in resultados:
    maximo += i 
    amostra_list_idx.extend(np.random.randint(low=mininimo, high=maximo,size=qtde_por_resultado))
    mininimo += i

#Usa os índices criados para gerar o banco de amostra
amostras = licoes.iloc[amostra_list_idx]        

def calcular_fit_creditus(genes):
    genes = np.array(genes)
    nac = 0
    nic = 0
    ta = 0
    ti = 0
    #db()  
    for i in range(amostras.shape[0]):         
        licao = amostras.iloc[i]
        lic_attr = licao[:atributos_qtde] 
        resultado = licao[-1]
        somaprod =(lic_attr* genes).values.sum()
        if somaprod > 0 and resultado == 'A':
            nac += 1
        elif somaprod < 0 and resultado == 'I':
            nic += 1
        if resultado == 'A':
            ta += 1
        else:
            ti += 1
   # fit = (nac*nic)/(ta*ti)
    fit = (nac+nic)/(ta+ti)
    return fit

### Creditus: execução da primeira geração

In [ ]:
rx = RandomX()
ga = Gapy(calcular_fit_creditus,rx.random,atributos_qtde)
ga.executar_por_geracao(1)
for c in ga.melhores_cromos():
    print('Id:', c.id, " / Fit: ",c.fit)

In [ ]:
for i in range(100):
    ga.executar_por_geracao(1)
    print(dp)
    for c in ga.melhores_cromos():
        print('Id:', c.id, " / Fit: ",c.fit)

In [ ]:
for i in range(1000):
    dp = ga.executar_por_geracao(1)
    c = ga.melhores_cromos()
    print(dp)    
    print('Id:', c[0].id, " / Fit: ", c[0].fit)

In [ ]:
rx = RandomX()
ga = Gapy(calcular_fit_creditus,rx.random,atributos_qtde,10,1,2)
ga.executar_por_rodada()
c = ga.melhores_cromos()
print(dp)    
print('Id:', c[0].id, " / Fit: ", c[0].fit)

In [ ]:
rx = RandomX()
ga = Gapy(calcular_fit_creditus,rx.random,atributos_qtde,60,1,2)
ga.executar_por_rodada()
c = ga.melhores_cromos()
print(dp)    
print_cromo(c)

In [ ]:
import random
random.seed(2)
random.random() 

In [ ]:
l = np.arange(0.01, 0.11, 0.01)
l

In [ ]:
l2 = np.arange(0.02, 0.21, 0.01)
l2

In [ ]:
l3 = list(l)
l3

In [ ]:
l3.append(list(l2))
l3

In [ ]:
l3 = list(l)
l3.extend(list(l2))
l3